In [16]:
! ollama pull wizardcoder

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling f890c9d2e6d4... 100% ▕████████████████▏ 3.8 GB                         
pulling 2d836d77287d... 100% ▕████████████████▏   61 B                         
pulling 8fadb9ad1206... 100% ▕████████████████▏  106 B                         
pulling 58e1b82a691f... 100% ▕████████████████▏   18 B                         
pulling c2623aa27c1e... 100% ▕████████████████▏  455 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling f890c9d2e6d4... 100% ▕████████████████▏ 3.8 GB                         
pulling 2d836d77287d... 100% ▕████████████████▏   61 B                         
pulling 8fadb9ad1206... 100% ▕████████████████▏  106 B                         
pulling 58e1b82a691f... 100% ▕████████████████▏   1

In [17]:
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
#from langchain.agents import AgentExecutor
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate


llm = Ollama(model="wizardcoder")
db = SQLDatabase.from_uri("sqlite:////Users/anders/SQLite/LocalSnowflakeMini.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer, always helpful, friendly and short and precise in your answrs."),
    ("user", "{input}")
])

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# run the chain
chain = prompt | agent_executor

chain.invoke({"input": "How many rows are there in the DISTRIBUTOR table?"})



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: ABT_FIXED_STOCK_COLL, ABT_FIXED_STOCK_IND, DISTRIBUTOR, VALID_ADDRESS
Thought:Question: messages=[SystemMessage(content='You are world class technical documentation writer, always helpful, friendly and short and precise in your answrs.'), HumanMessage(content='How many rows are there in the DISTRIBUTOR table?')]
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: ""
Observation: ABT_FIXED_STOCK_COLL, ABT_FIXED_STOCK_IND, DISTRIBUTOR, VALID_ADDRESS
Thought:Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: ""
Observation: ABT_FIXED_STOCK_COLL, ABT_FIXED_STOCK_IND, DISTRIBUTOR, VALID_ADDRESS
Thought:Thought: I should look at the tables in t

KeyboardInterrupt: 

In [4]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [5]:
agent_executor.invoke('How many rows are in the DISTRIBUTOR table?')

/Users/anders/miniconda3/envs/DataChatter/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Action: sql_db_list_tables
Input: `

In [31]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [4]:
chain.invoke("Are you good at writing SQL for Snowflake?")

'Yes, I am quite experienced in using and creating SQL queries in Snowflake.\r\n\r\nSure, I can help you with your project! '

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer, always helpful, friendly and short and precise in your answrs."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm

In [7]:
chain.invoke({"input": "How well do you know the Snowflake documentation?"})

'I am familiar with most of the Snowflake documentation, but I have to admit that it is always better to refer to the official documentation for specific questions or answers as they are constantly updated and accurate. I can help you find the information you need by referring to their website or support channels, such as community forums, or reach out to a technical expert if needed. '

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser

In [12]:
chain.invoke({"input": "How well do you know the Streamlit chat functionality?"})

"I am familiar with the basics of Streamlit's chat functionality. I have used it for prototyping small applications that require user inputs from users, but not extensively enough to be an expert on its features and functions. However, I can help you with any specific questions or queries related to it. "

In [11]:
#! pip install beautifulsoup4

In [13]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.streamlit.io/library/api-reference/chat/st.chat_message")

docs = loader.load()

In [14]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [17]:
! pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 14.3 MB/s eta 0:00:0000:0100:01


In [15]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [17]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [18]:
response = retrieval_chain.invoke({"input": "do you know the documentation for Streamlits st.chat_message?"})
print(response["answer"])

The documentation for `st.chat_message` is available on the [Streamlit website](https://streamlit.io/docs/api.html). Here's a link to the specific section related to the `chat_message` function: https://streamlit.io/docs/api.html#streamlit.chat_message 


In [19]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [20]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="How can I find information about how to make a streamlit app?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='st.chat_message - Streamlit DocsDocumentationsearchSearchrocket_launchGet startedInstallationaddFundamentalsaddTutorialsadddescriptionStreamlit libraryAPI referenceremoveWrite and magicaddText elementsaddData elementsaddChart elementsaddInput widgetsaddMedia elementsaddLayouts and containersaddChat elementsremovest.chat_messagest.chat_inputStatus elementsaddControl flowaddUtilitiesaddMutate chartsState managementPerformanceaddPersonalizationaddConnections and databasesaddApp testingaddAdvanced featuresaddComponentsaddRoadmapopen_in_newChangelogCheat sheetcloudStreamlit Community CloudGet startedaddDeploy your appaddManage your appaddShare your appaddManage your accountaddTroubleshootingschoolKnowledge baseTutorialsaddUsing StreamlitStreamlit ComponentsInstalling dependenciesDeployment issuesHome/Streamlit library/API reference/Chat elements/st.chat_messagestarTipRead the Build a basic LLM chat app tutorial to learn how to use st.chat_message and st.chat_input to

In [21]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [22]:
chat_history = [HumanMessage(content="In one word, do you know enough of the Streamlit documentation to help me build a Langchain app with Streamlit?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='In one word, do you know enough of the Streamlit documentation to help me build a Langchain app with Streamlit?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='st.chat_message - Streamlit DocsDocumentationsearchSearchrocket_launchGet startedInstallationaddFundamentalsaddTutorialsadddescriptionStreamlit libraryAPI referenceremoveWrite and magicaddText elementsaddData elementsaddChart elementsaddInput widgetsaddMedia elementsaddLayouts and containersaddChat elementsremovest.chat_messagest.chat_inputStatus elementsaddControl flowaddUtilitiesaddMutate chartsState managementPerformanceaddPersonalizationaddConnections and databasesaddApp testingaddAdvanced featuresaddComponentsaddRoadmapopen_in_newChangelogCheat sheetcloudStreamlit Community CloudGet startedaddDeploy your appaddManage your appaddShare your appaddManage your accountaddTroubleshootingschoolKnowledge baseTutorialsaddUsing StreamlitStreamlit Co

In [23]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "streamlit_search",
    "Search for information about Streamlit. For any questions about Streamlit, you must use this tool!",
)

In [28]:
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType

In [45]:
db = SQLDatabase.from_uri("sqlite:////Users/anders/SQLite/LocalSnowflakeMini.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [46]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [49]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [51]:
# chain together the agent and the output parser
chain = agent_executor | output_parser

# run the chain
chain.invoke("how many addresses are there?")

#agent_executor.run("how many addresses are there?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: ABT_FIXED_STOCK_COLL, ABT_FIXED_STOCK_IND, DISTRIBUTOR, VALID_ADDRESS
Thought:

KeyboardInterrupt: 